# Method Comparison: G-computation, IPW, and AIPW

This notebook compares different causal inference methods on the same dataset to demonstrate their relative strengths and weaknesses.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from causal_inference.core import CovariateData, OutcomeData, TreatmentData
from causal_inference.estimators import AIPW, IPW, GComputation

# Set up plotting
plt.style.use("default")
plt.rcParams["figure.figsize"] = (10, 6)

## Generate Data with Known Treatment Effect

We'll create data where we know the true causal effect is 0.15.

In [ ]:
np.random.seed(123)
n = 2000

# Confounders
X1 = np.random.normal(0, 1, n)
X2 = np.random.normal(0, 1, n)
X3 = np.random.binomial(1, 0.5, n)

# Treatment assignment (depends on confounders)
treatment_logit = -0.5 + 0.5 * X1 + 0.3 * X2 + 0.4 * X3
treatment_prob = 1 / (1 + np.exp(-treatment_logit))
treatment = np.random.binomial(1, treatment_prob, n)

# Outcome (depends on both treatment and confounders)
# TRUE CAUSAL EFFECT = 0.15
outcome_logit = -1 + 0.15 * treatment + 0.4 * X1 + 0.3 * X2 + 0.2 * X3
outcome_prob = 1 / (1 + np.exp(-outcome_logit))
outcome = np.random.binomial(1, outcome_prob, n)

# Create DataFrame
data = pd.DataFrame(
    {"X1": X1, "X2": X2, "X3": X3, "treatment": treatment, "outcome": outcome}
)

print(f"Sample size: {n}")
print(f"Treatment rate: {treatment.mean():.3f}")
print(f"Outcome rate: {outcome.mean():.3f}")
print("True causal effect: 0.150")

## Set up Data Objects

In [ ]:
# Define causal inference data objects
treatment_data = TreatmentData(
    values=data["treatment"], name="treatment", treatment_type="binary"
)

outcome_data = OutcomeData(
    values=data["outcome"], name="outcome", outcome_type="binary"
)

covariate_data = CovariateData(
    values=data[["X1", "X2", "X3"]], names=["X1", "X2", "X3"]
)

## Compare Methods

In [ ]:
# Initialize estimators
methods = {
    "G-computation": GComputation(),
    "IPW": IPW(),
    "AIPW (Doubly Robust)": AIPW(),
}

results = {}

# Estimate effects with each method
for name, estimator in methods.items():
    effect = estimator.estimate_ate(
        treatment=treatment_data, outcome=outcome_data, covariates=covariate_data
    )

    results[name] = {
        "ate": effect.ate,
        "se": effect.se,
        "ci_lower": effect.ci_lower,
        "ci_upper": effect.ci_upper,
        "p_value": effect.p_value,
    }

# Also compute naive difference
treated_mean = data[data["treatment"] == 1]["outcome"].mean()
control_mean = data[data["treatment"] == 0]["outcome"].mean()
naive_diff = treated_mean - control_mean

results["Naive Difference"] = {
    "ate": naive_diff,
    "se": np.nan,
    "ci_lower": np.nan,
    "ci_upper": np.nan,
    "p_value": np.nan,
}

## Results Comparison

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results).T
results_df["bias"] = results_df["ate"] - 0.150  # True effect is 0.15
results_df["abs_bias"] = np.abs(results_df["bias"])

print("📊 Method Comparison Results")
print("=" * 50)
print(f"{'Method':<20} {'ATE':<8} {'SE':<8} {'95% CI':<20} {'Bias':<8}")
print("-" * 50)

for method, row in results_df.iterrows():
    ate_str = f"{row['ate']:.4f}"
    se_str = f"{row['se']:.4f}" if not pd.isna(row["se"]) else "N/A"
    ci_str = (
        f"[{row['ci_lower']:.3f}, {row['ci_upper']:.3f}]"
        if not pd.isna(row["ci_lower"])
        else "N/A"
    )
    bias_str = f"{row['bias']:.4f}"

    print(f"{method:<20} {ate_str:<8} {se_str:<8} {ci_str:<20} {bias_str:<8}")

print("\nTrue causal effect: 0.150")

## Visualize Results

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Point estimates with confidence intervals
methods_plot = [m for m in results_df.index if m != "Naive Difference"]
ates = [results_df.loc[m, "ate"] for m in methods_plot]
ses = [results_df.loc[m, "se"] for m in methods_plot]
ci_lowers = [results_df.loc[m, "ci_lower"] for m in methods_plot]
ci_uppers = [results_df.loc[m, "ci_upper"] for m in methods_plot]

y_pos = range(len(methods_plot))

ax1.errorbar(
    ates,
    y_pos,
    xerr=[np.array(ates) - np.array(ci_lowers), np.array(ci_uppers) - np.array(ates)],
    fmt="o",
    capsize=5,
    capthick=2,
    markersize=8,
)

# Add true effect line
ax1.axvline(x=0.150, color="red", linestyle="--", linewidth=2, label="True Effect")

# Add naive estimate
ax1.axvline(
    x=naive_diff, color="gray", linestyle=":", linewidth=2, label="Naive Difference"
)

ax1.set_yticks(y_pos)
ax1.set_yticklabels(methods_plot)
ax1.set_xlabel("Average Treatment Effect")
ax1.set_title("ATE Estimates with 95% Confidence Intervals")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Bias comparison
all_methods = list(results_df.index)
biases = [results_df.loc[m, "bias"] for m in all_methods]

colors = [
    "blue"
    if "AIPW" in m
    else "green"
    if m == "G-computation"
    else "orange"
    if m == "IPW"
    else "red"
    for m in all_methods
]

bars = ax2.bar(range(len(all_methods)), biases, color=colors, alpha=0.7)
ax2.axhline(y=0, color="black", linestyle="-", linewidth=1)
ax2.set_xticks(range(len(all_methods)))
ax2.set_xticklabels(
    [m.replace(" ", "\n") for m in all_methods], rotation=45, ha="right"
)
ax2.set_ylabel("Bias (Estimate - True Effect)")
ax2.set_title("Bias Comparison")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Key Takeaways

1. **Naive Difference** is biased due to confounding
2. **G-computation** works well when the outcome model is correct
3. **IPW** works well when the propensity model is correct
4. **AIPW** is "doubly robust" - works if either model is correct

In practice, AIPW is often preferred because it provides protection against model misspecification.

## Sensitivity Analysis

Let's see how sensitive our results are to model specification.

In [ ]:
# Test with different model complexities
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_configs = {
    "Linear Models": {
        "outcome_model": LogisticRegression(),
        "propensity_model": LogisticRegression(),
    },
    "Random Forest": {
        "outcome_model": RandomForestClassifier(n_estimators=50, random_state=42),
        "propensity_model": RandomForestClassifier(n_estimators=50, random_state=42),
    },
}

print("🔍 Sensitivity to Model Choice")
print("=" * 40)

for config_name, models in model_configs.items():
    # Test AIPW with different models
    aipw = AIPW(**models)
    effect = aipw.estimate_ate(
        treatment=treatment_data, outcome=outcome_data, covariates=covariate_data
    )

    bias = effect.ate - 0.150
    print(f"{config_name}: ATE = {effect.ate:.4f}, Bias = {bias:.4f}")